In [1]:
import pandas as pd
import os
import numpy as np
import requests
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series



In [2]:
import hsfs

In [3]:
import hopsworks

In [4]:
load_dotenv()
entose_api = os.getenv("ENTSOE_API")
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
print(f"Connected to project: {project.name}")

2025-01-06 09:41:08,052 INFO: Initializing external client
2025-01-06 09:41:08,053 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 09:41:10,071 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [7]:
start_date = "2022-11-01"
end_date = "2025-01-06"
end_date = datetime.date.today().strftime('%Y-%m-%d')
hist_weather_df = get_historical_weather("Stockhom", "2022-11-01", "2025-01-06", 59.3294, 18.0687)
formatted_hist_weather_df = process_weather_data(hist_weather_df)

2022-11-01
2025-01-06
Coordinates 59.29701232910156°N 18.163265228271484°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
MAX DATE
2024-12-31


In [8]:
#print(formatted_hist_weather_df.head())
#print(formatted_hist_weather_df.dtypes)
print(formatted_hist_weather_df['date'].tail(1))

print(formatted_hist_weather_df['date'].max())

19007   2024-12-31 23:00:00
Name: date, dtype: datetime64[ns]
2024-12-31 23:00:00


In [9]:
entsoe_df = fetch_historical_data(entose_api, start_date, end_date)

[DEBUG] Successfully fetched load data for finland.
[DEBUG] Successfully fetched load data for norway.
[DEBUG] Successfully fetched load data for denmark.
[DEBUG] Successfully fetched cross-border flows SE3 to finland.
[DEBUG] Successfully fetched cross-border flows finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to norway.
[DEBUG] Successfully fetched cross-border flows norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to denmark.
[DEBUG] Successfully fetched cross-border flows denmark to SE3.


In [10]:
#print(formatted_hist_weather_df.head())
#print(entsoe_df.head())
entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_df.columns = entsoe_df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])

In [11]:
#print(entsoe_df.dtypes)
print(entsoe_df['date'].tail(1))

index
2025-01-05 22:00:00+00:00   2025-01-05 23:00:00
Name: date, dtype: datetime64[ns]


In [14]:
merged_df = pd.merge(formatted_hist_weather_df, entsoe_df, on='date', how='inner')

In [15]:
print(merged_df['date'].max())

2024-12-31 23:00:00


In [16]:
print(entsoe_df.dtypes)

load_se3                                                   float64
load_finland                                               float64
load_norway                                                float64
load_denmark                                               float64
prices                                                     float64
total_generation_biomass                                   float64
total_generation_fossil_gas                                float64
total_generation_fossil_hard_coal                          float64
total_generation_fossil_oil                                float64
total_generation_hydro_run_of_river_and_poundage           float64
total_generation_other_renewable                           float64
total_generation_solar                                     float64
total_generation_waste                                     float64
total_generation_wind_offshore                             float64
total_generation_wind_onshore                              flo

In [ ]:
#entsoe_df['date'] = pd.to_datetime(entsoe_df['date']).dt.tz_convert(None)
#entsoe_df['date'] = pd.to_datetime(entsoe_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)
#formatted_hist_weather_df['date'] = pd.to_datetime(formatted_hist_weather_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)

In [19]:

print(merged_df.dtypes)
print(merged_df['date'].max())

date                                                datetime64[ns]
temperature_2m                                             float32
precipitation                                              float32
snow_depth                                                 float32
pressure_msl                                               float32
cloud_cover                                                float32
wind_speed_10m                                             float32
wind_speed_100m                                            float32
wind_direction_10m                                         float32
wind_direction_100m                                        float32
city                                                        object
load_se3                                                   float64
load_finland                                               float64
load_norway                                                float64
load_denmark                                               flo

In [20]:
print(formatted_hist_weather_df.dtypes)

date                   datetime64[ns]
temperature_2m                float32
precipitation                 float32
snow_depth                    float32
pressure_msl                  float32
cloud_cover                   float32
wind_speed_10m                float32
wind_speed_100m               float32
wind_direction_10m            float32
wind_direction_100m           float32
city                           object
dtype: object


In [11]:
# price_df = get_data("SE3")
# print(price_df.dtypes)
# print(price_df)

In [15]:
# price_df['time_start'] = price_df['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
# print(price_df)
# print(price_df.dtypes)

In [13]:
# formatted_hist_weather_df['time_start'] = formatted_hist_weather_df['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
# print(formatted_hist_weather_df)
# print(formatted_hist_weather_df.dtypes)


In [14]:
# print(price_df)
# print(price_df.dtypes)

In [17]:

# print(price_df.head())
# print(price_df.dtypes)

In [21]:
import great_expectations as ge

weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"temperature_2m",
            "min_value":-100.0,
            "max_value":500.0,
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"precipitation",
            "min_value":-5.0,
            "max_value":500.0,

        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pressure_msl",
            "min_value":0.0,
            "max_value":2000,
        }
    )
)



weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"cloud_cover",
            "min_value":-0.1,
            "max_value":100.0,
            "strict_min":True
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"wind_speed_10m",
            "min_value":-0.1,
            "max_value":1000,
            "strict_min":True
        }
    )
)



{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "wind_speed_10m", "min_value": -0.1, "max_value": 1000, "strict_min": true}, "meta": {}}

In [9]:
# import great_expectations as ge

# prices_expectation_suite = ge.core.ExpectationSuite(
#     expectation_suite_name="price_expectation_suite"
# )

# prices_expectation_suite.add_expectation(
#     ge.core.ExpectationConfiguration(
#         expectation_type="expect_column_min_to_be_between",
#         kwargs={
#             "column":"price",
#             "min_value":-0.1,
#             "max_value":5000,
#             "strict_min":True
#         }
#     )
# )

In [22]:
import great_expectations as ge

data_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="data_expectation_suite"
)
columns = [
    'load_se3', 'load_finland', 'load_norway', 'load_denmark', 'prices',
    'total_generation_biomass', 'total_generation_fossil_gas',
    'total_generation_fossil_hard_coal', 'total_generation_fossil_oil',
    'total_generation_hydro_run_of_river_and_poundage',
    'total_generation_other_renewable', 'total_generation_solar',
    'total_generation_waste', 'total_generation_wind_offshore',
    'total_generation_wind_onshore', 'flows_se3_to_finland',
    'flows_finland_to_se3', 'flows_se3_to_norway', 'flows_norway_to_se3',
    'flows_se3_to_denmark', 'flows_denmark_to_se3', 'date'
]

# Define min and max values for the expectations
min_max_values = {
    'load_se3': (-0.1, 20000),
    'load_finland': (-0.1, 20000),
    'load_norway': (-0.1, 20000),
    'load_denmark': (-0.1, 20000),
    'prices': (-1000, 1000),
    'total_generation_biomass': (-0.1, 10000),
    'total_generation_fossil_gas': (-0.1, 10000),
    'total_generation_fossil_hard_coal': (-0.1, 10000),
    'total_generation_fossil_oil': (-0.1, 10000),
    'total_generation_hydro_run_of_river_and_poundage': (-0.1, 10000),
    'total_generation_other_renewable': (-0.1, 10000),
    'total_generation_solar': (-0.1, 10000),
    'total_generation_waste': (-0.1, 10000),
    'total_generation_wind_offshore': (-0.1, 10000),
    'total_generation_wind_onshore': (-0.1, 10000),
    'flows_se3_to_finland': (-0.1, 10000),
    'flows_finland_to_se3': (-0.1, 10000),
    'flows_se3_to_norway': (-0.1, 10000),
    'flows_norway_to_se3': (-0.1, 10000),
    'flows_se3_to_denmark': (-0.1, 10000),
    'flows_denmark_to_se3': (-0.1, 10000)
}

# Add expectations for each column
for column, (min_value, max_value) in min_max_values.items():
    data_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column": column,
                "min_value": min_value,
                "max_value": max_value,
                "strict_min": True
            }
        )
    )

    print(data_expectation_suite)

{
  "expectation_suite_name": "data_expectation_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "load_se3",
        "min_value": -0.1,
        "max_value": 20000,
        "strict_min": true
      },
      "meta": {}
    }
  ],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}
{
  "expectation_suite_name": "data_expectation_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "load_se3",
        "min_value": -0.1,
        "max_value": 20000,
        "strict_min": true
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "load_finland",
        "min_value": -0.1,
        "max_value": 20000,
        "strict_min": true
      },
      "meta": {}
    }
  ],
  "data_ass

In [23]:
try:
    fs = project.get_feature_store()
    print(f"Connected to feature store: {fs.name}")
except Exception as e:
    print(f"Failed to get feature store: {e}")

Connected to feature store: oskaralf_featurestore


In [24]:
weather_fg = fs.get_or_create_feature_group(
    name='weather_data_3',
    description='Weather data for SE3',
    version=1,
    primary_key=['date'],
    event_time='date', 
    expectation_suite=weather_expectation_suite
)

In [25]:
entsoe_fg = fs.get_or_create_feature_group(
    name='entsoe_data_3',
    description='Entsoe data for SE3',
    version=1,
    primary_key=['date'],
    event_time='date',
    expectation_suite=data_expectation_suite
)

In [ ]:
# price_fg = fs.get_or_create_feature_group(
#     name='electricity_price_data_3',
#     description='Electricity price data for SE3',
#     version=1,
#     primary_key=['time_start'],
#     expectation_suite=prices_expectation_suite
# )

In [26]:
formatted_hist_weather_df = formatted_hist_weather_df.dropna()
weather_fg.insert(formatted_hist_weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394632
2025-01-06 09:49:44,169 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394632


Uploading Dataframe: 100.00% |██████████| Rows 18504/18504 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_3_1_offline_fg_materialization/executions


(Job('weather_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pressure_msl",
           "min_value": 0.0,
           "max_value": 2000
         },
         "meta": {
           "expectationId": 694524
         }
       },
       "result": {
         "observed_value": 962.7999877929688,
         "element_count": 18504,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T08:49:44.000168Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betw

In [27]:
entsoe_df = entsoe_df.dropna()
entsoe_fg.insert(entsoe_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394633
2025-01-06 09:50:03,115 INFO: 	21 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394633


Uploading Dataframe: 100.00% |██████████| Rows 19123/19123 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: entsoe_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_3_1_offline_fg_materialization/executions


(Job('entsoe_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "flows_denmark_to_se3",
           "min_value": -0.1,
           "max_value": 10000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 694543
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 19123,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T08:50:03.000114Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "ex